In [13]:
from recordclass import dataobject, make_dataclass, enable_gc, litelist


In [9]:
from random import randint
import tracemalloc
class Tracer:
    def __enter__(self):
        if tracemalloc.is_tracing():
            raise ValueError('nesting tracemalloc is not allowed')
        self.allocated = None
        tracemalloc.start()
        return self
    def __exit__(self, exc_type, exc_value, exc_traceback):
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        self.allocated = current

In [5]:
N = 22
print(2**N/1000000, "млн.")

4.194304 млн.


### Binary Tree

In [14]:
class Tree(dataobject):
   __fields__ = 'root',

class Node(dataobject): 
    __fields__ = 'left', 'right'

class Leaf(dataobject):
    __fields__ = 'value',

In [15]:
def add_nodes(depth):
    if depth == 0:
        return Leaf(randint(0,10))
    return Node(add_nodes(depth-1), add_nodes(depth-1))

In [ ]:
@enable_gc
class TreeGC(dataobject):
   __fields__ = 'root',

@enable_gc
class NodeGC(dataobject): 
    __fields__ = 'left', 'right'

@enable_gc
class LeafGC(dataobject):
    __fields__ = 'value',

In [ ]:
def add_nodes_gc(depth):
    if depth == 0:
        return LeafGC(randint(0,10))
    return NodeGC(add_nodes_gc(depth-1), add_nodes_gc(depth-1))

In [1]:
class TreeSlots:
    __slots__ = 'root',
    def __init__(self, root):
        self.root = root

class NodeSlots:
    __slots__ = 'left', 'right'
    def __init__(self, left, right):
        self.left = left
        self.right = right
    
class LeafSlots:
    __slots__ = 'value',
    def __init__(self, value):
        self.value = value

In [7]:
def add_nodes_slots(depth):
    if depth == 0:
        return LeafSlots(randint(0,10))
    return NodeSlots(add_nodes_slots(depth-1), add_nodes_slots(depth-1))

In [16]:
with Tracer() as t1:
   tree = Tree(add_nodes(N))
print("dataobject:", t1.allocated // 1_000_000, 'Mb')
del tree, t1

dataobject: 234 Mb


In [17]:
%time tree = Tree(add_nodes(N))
%time del tree

CPU times: user 5.7 s, sys: 79.9 ms, total: 5.78 s
Wall time: 5.78 s
CPU times: user 163 ms, sys: 16 ms, total: 179 ms
Wall time: 179 ms


In [18]:
with Tracer() as t2:
   tree_slots = TreeSlots(add_nodes_slots(N))
print("__slots__:", t2.allocated // 1_000_000, 'Mb')
del tree_slots, t2

__slots__: 503 Mb


In [ ]:
%time tree_slots = TreeSlots(add_nodes_slots(N))
%time del tree_slots

In [ ]:
with Tracer() as t3:
   tree_gc = TreeGC(add_nodes_gc(N))
print("dataobject+gc:", t3.allocated // 1000000, 'Mb')
del tree_gc, t3

In [ ]:
%time tree_gc = TreeGC(add_nodes_gc(N))
%time del tree_gc

In [ ]:
import sys
print('Size per instance:')
print('   node:', sys.getsizeof(Node(None,None)),
                  sys.getsizeof(NodeSlots(None,None)), 
                  sys.getsizeof(NodeGC(None,None)))
print('   leaf:', sys.getsizeof(Leaf(None)),
                  sys.getsizeof(LeafSlots(None)),
                  sys.getsizeof(LeafGC(None)))

### General Tree

In [ ]:
class GTree(dataobject):
   __fields__ = 'root',

class GNode(dataobject): 
    __fields__ = 'children',

class GLeaf(dataobject):
    __fields__ = 'value',

In [ ]:
def add_gnodes(depth):
    if depth == 0:
        return GLeaf(randint(0,10))
    children = litelist([])
    gnode = GNode(children)
    children.append(add_gnodes(depth-1))
    children.append(add_gnodes(depth-1))
    children.trim()
    return gnode

In [ ]:
with Tracer() as gt1:
   gtree = GTree(add_gnodes(20))
print("dataobject:", gt1.allocated // 1000000, 'Mb')
del gtree, gt1

In [ ]:
%time gtree = GTree(add_gnodes(20))
del gtree

In [ ]:
class GTreeSlots:
    __slots__ = 'root',
    def __init__(self, root):
        self.root = root

class GNodeSlots:
    __slots__ = 'children', 
    def __init__(self, children):
        self.children = children
    
class GLeafSlots:
    __slots__ = 'value',
    def __init__(self, value):
        self.value = value

In [ ]:
def add_gnodes_slots(depth):
    if depth == 0:
        return GLeafSlots(randint(0,10))
    children = []
    gnode = GNodeSlots(children)
    children.append(add_gnodes_slots(depth-1))
    children.append(add_gnodes_slots(depth-1))
    return gnode

In [ ]:
with Tracer() as gt2:
   gtree_slots = GTreeSlots(add_gnodes_slots(20))
print("__slots__:", gt2.allocated // 1000000, 'Mb')
del gtree_slots, gt2

In [ ]:
%time gtree_slots = GTreeSlots(add_gnodes_slots(20))
del gtree_slots